<a href="https://colab.research.google.com/github/cloverbunny/mimic-iv/blob/master/c14_combine_mimic_files3_20_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cleaning mimic files so far into one notebook

Notes from Notebook on how to run bigquery in colab - https://github.com/MIT-LCP/mimic-code/discussions/1154

# Setup

Load libraries and connect to the data

Run the following cells to import some libraries and then connect to the database.

In [1]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [11]:
auth.authenticate_user()

In [3]:
project_id='prismatic-fact-272817'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

# Getting intubated patients who stayed in MICU units (includes units that are MICU/SICU)

[BigQuery syntax](https://googleapis.dev/python/bigquery/latest/magics.html).

In [43]:
%%bigquery vented

select *
from `physionet-data.mimic_derived.ventilation` v join  `physionet-data.mimic_icu.icustays` icus on v.stay_id = icus.stay_id
where ventilation_status like 'InvasiveVent'
AND
  first_careunit LIKE '%MICU%'
  AND last_careunit LIKE '%MICU%'


In [44]:
vented.shape

(10915, 12)

In [45]:
vented.head()

,stay_id,starttime,endtime,ventilation_status,subject_id,hadm_id,stay_id_1,first_careunit,last_careunit,intime,outtime,los
0,31326208,2135-06-20 23:18:00,2135-06-22 01:00:00,InvasiveVent,19120385,23418362,31326208,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2135-06-20 23:00:00,2135-06-22 06:02:12,1.293194
1,31004416,2120-02-09 11:00:00,2120-02-10 08:00:00,InvasiveVent,13390728,26350081,31004416,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2120-02-06 20:55:28,2120-02-29 21:46:04,23.035139
2,31004416,2120-02-24 03:00:00,2120-02-24 15:00:00,InvasiveVent,13390728,26350081,31004416,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2120-02-06 20:55:28,2120-02-29 21:46:04,23.035139
3,31004416,2120-02-12 15:00:00,2120-02-13 12:40:00,InvasiveVent,13390728,26350081,31004416,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2120-02-06 20:55:28,2120-02-29 21:46:04,23.035139
4,31004416,2120-02-06 21:39:00,2120-02-07 08:00:00,InvasiveVent,13390728,26350081,31004416,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2120-02-06 20:55:28,2120-02-29 21:46:04,23.035139


In [48]:
vented_unique = vented.drop_duplicates(subset=["hadm_id"])

In [49]:
vented_unique.shape

(7042, 12)

## Getting pneumonia patients

In [27]:
%%bigquery pneumonia

SELECT *
from `physionet-data.mimic_hosp.diagnoses_icd` dx
join `physionet-data.mimic_hosp.d_icd_diagnoses` icd
    on dx.icd_code=icd.icd_code
where icd.long_title LIKE '%pneumonia%'

In [28]:
pneumonia.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,icd_code_1,icd_version_1,long_title
0,17967970,21746242,31,B953,10,B953,10,Streptococcus pneumoniae as the cause of disea...
1,18622276,20636275,25,B953,10,B953,10,Streptococcus pneumoniae as the cause of disea...
2,13610988,25620263,31,J1289,10,J1289,10,Other viral pneumonia
3,10011849,20188892,36,B953,10,B953,10,Streptococcus pneumoniae as the cause of disea...
4,12227782,29222288,1,48249,9,48249,9,Other Staphylococcus pneumonia


In [29]:
pneumonia.shape

(10673, 8)

In [30]:
len(pneumonia.subject_id.unique())

8584

In [31]:
len(pneumonia.hadm_id.unique())

9999

In [34]:
pneumonia.long_title.value_counts()

Ventilator associated pneumonia                                                                                           1723
Klebsiella pneumoniae [K. pneumoniae] as the cause of diseases classified elsewhere                                       1425
Personal history of pneumonia (recurrent)                                                                                 1208
Bacterial pneumonia, unspecified                                                                                          1093
Other specified vaccinations against streptococcus pneumoniae [pneumococcus]                                              1007
Unspecified bacterial pneumonia                                                                                            716
Methicillin resistant pneumonia due to Staphylococcus aureus                                                               346
Methicillin susceptible pneumonia due to Staphylococcus aureus                                                 

In [33]:
pd.set_option("display.max_rows", None)

In [35]:
names_to_remove = [
    'Other specified vaccinations against streptococcus pneumoniae [pneumococcus]',
    'Need for prophylactic vaccination and inoculation against streptococcus pneumoniae [pneumococcus] and influenza',
    'Cryptogenic organizing pneumonia',
    'Congenital pneumonia',
    'Abscess of lung without pneumonia',    
    'Congenital pneumonia, unspecified',
    'Congenital pneumonia due to staphylococcus',
    'Idiopathic interstitial pneumonia, not otherwise specified',
    'Congenital pneumonia due to other bacterial agents',
    'Desquamative interstitial pneumonia',
    'Congenital pneumonia due to Escherichia coli',
    'Infection by Histoplasma capsulatum, pneumonia',
    'Tuberculous pneumonia [any form], tubercle bacilli found (in sputum) by microscopy',
    'Whooping cough, unspecified species without pneumonia',
    'Congenital pneumonia due to Pseudomonas',
    'Congenital pneumonia due to viral agent',
    'Hypostatic pneumonia, unspecified organism',
    'Lymphoid interstitial pneumonia',
    'Congenital pneumonia due to streptococcus, group B',
    'Tuberculous pneumonia [any form], tubercle bacilli not found (in sputum) by microscopy, but found by bacterial culture',
    'Tuberculous pneumonia [any form], unspecified',  
    'Idiopathic lymphoid interstitial pneumonia'  
]

In [36]:
pneumonia_clean= pneumonia[~pneumonia['long_title'].isin(names_to_remove)]

In [37]:
pneumonia_clean.long_title.value_counts()

Ventilator associated pneumonia                                                                               1723
Klebsiella pneumoniae [K. pneumoniae] as the cause of diseases classified elsewhere                           1425
Personal history of pneumonia (recurrent)                                                                     1208
Bacterial pneumonia, unspecified                                                                              1093
Unspecified bacterial pneumonia                                                                                716
Methicillin resistant pneumonia due to Staphylococcus aureus                                                   346
Methicillin susceptible pneumonia due to Staphylococcus aureus                                                 341
Pneumonia due to Klebsiella pneumoniae                                                                         276
Influenza with pneumonia                                                        

In [41]:
pneumonia_clean.shape

(9084, 8)

In [38]:
len(pneumonia_clean.hadm_id.unique())

8459

In [39]:
pneumonia_short = pneumonia_clean.drop_duplicates(subset=["hadm_id"])

In [40]:
pneumonia_short.shape

(8459, 8)

# Merging pneumonia and vented MICU patients

In [50]:
micuventpneumonia = vented_unique.merge(pneumonia_short, how='inner', on='hadm_id', indicator=True)

In [51]:
micuventpneumonia.head()

,stay_id,starttime,endtime,ventilation_status,subject_id_x,hadm_id,stay_id_1,first_careunit,last_careunit,intime,outtime,los,subject_id_y,seq_num,icd_code,icd_version,icd_code_1,icd_version_1,long_title,_merge
0,31004416,2120-02-09 11:00:00,2120-02-10 08:00:00,InvasiveVent,13390728,26350081,31004416,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2120-02-06 20:55:28,2120-02-29 21:46:04,23.035139,13390728,8,J181,10,J181,10,"Lobar pneumonia, unspecified organism",both
1,39008512,2136-09-14 21:00:00,2136-09-15 00:00:00,InvasiveVent,17521905,29059095,39008512,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2136-09-02 15:09:45,2136-09-23 21:30:42,21.264549,17521905,8,J95851,10,J95851,10,Ventilator associated pneumonia,both
2,31813120,2151-05-25 14:00:00,2151-05-26 16:45:00,InvasiveVent,18081075,28510423,31813120,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2151-05-25 14:13:00,2151-06-07 14:32:42,13.013681,18081075,1,J1000,10,J1000,10,Influenza due to other identified influenza vi...,both
3,32863488,2171-06-04 10:00:00,2171-06-11 13:00:00,InvasiveVent,17914766,24743178,32863488,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2171-06-04 10:32:22,2171-06-17 22:30:20,13.498588,17914766,7,J95851,10,J95851,10,Ventilator associated pneumonia,both
4,35174400,2166-03-15 10:00:00,2166-03-19 07:00:00,InvasiveVent,13870531,23854031,35174400,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2166-03-10 19:01:30,2166-03-29 18:25:59,18.975336,13870531,5,4870,9,4870,9,Influenza with pneumonia,both


In [52]:
micuventpneumonia.shape

(1284, 20)

In [53]:
micuventpneumonia.to_csv("micuventpneumonia3-20-22.csv")

# Patients who had respiratory samples


In [54]:
%%bigquery respsamples

select hadm_id
from `physionet-data.mimic_hosp.microbiologyevents`
where spec_type_desc in 
("BRONCHOALVEOLAR LAVAGE",
"Mini-BAL",
"TRACHEAL ASPIRATE",
"BRONCHIAL WASHINGS",
"Rapid Respiratory Viral Screen & Culture"
)
    AND hadm_id is not NULL
group by hadm_id

In [56]:
respsamples.shape

(10603, 1)

In [62]:
#micu vent + samples
micuventsample = vented_unique.merge(respsamples, how='inner', on='hadm_id', indicator=True)

In [63]:
micuventsample.shape

(1859, 13)

In [64]:
micuventsample.head()

,stay_id,starttime,endtime,ventilation_status,subject_id,hadm_id,stay_id_1,first_careunit,last_careunit,intime,outtime,los,_merge
0,31004416,2120-02-09 11:00:00,2120-02-10 08:00:00,InvasiveVent,13390728,26350081,31004416,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2120-02-06 20:55:28,2120-02-29 21:46:04,23.035139,both
1,39663616,2117-01-02 20:00:00,2117-01-04 12:00:00,InvasiveVent,11051942,26447909,39663616,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2116-12-21 21:26:00,2117-01-07 19:52:12,16.934861,both
2,39008512,2136-09-14 21:00:00,2136-09-15 00:00:00,InvasiveVent,17521905,29059095,39008512,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2136-09-02 15:09:45,2136-09-23 21:30:42,21.264549,both
3,30950144,2189-07-05 22:00:00,2189-07-07 08:00:00,InvasiveVent,13387130,27400251,30950144,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-07-05 21:23:00,2189-07-07 14:14:13,1.702234,both
4,31813120,2151-05-25 14:00:00,2151-05-26 16:45:00,InvasiveVent,18081075,28510423,31813120,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2151-05-25 14:13:00,2151-06-07 14:32:42,13.013681,both


# MICU, vented, pneumonia dx, resp sample


In [65]:

micuventpneumoniasample = micuventpneumonia.merge(respsamples, how='inner', on='hadm_id')

In [67]:
micuventpneumoniasample.shape

(622, 20)

In [70]:
len(micuventpneumoniasample.stay_id.unique())

622